<a href="https://colab.research.google.com/github/deniseiras/EscolaVeraoSantosDumont2023/blob/main/taxi_cab_2020_map_JCD_MC_CD02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Yellow Taxi cab: registra viagens de taxi em Manhattan
![picture](https://drive.google.com/uc?export=view&id=1hoFUFSm5IJG-5CQYBtqDK_e2MvMODc1P)
## Este notebook explora o processamento de dados tabulares (i.e viagens de taxi)
## Neste exemplo, usamos a API RDD.
## Procuramos por viagens não muito vantajosas, como muitos passageiros e baixa tarifa :)

##site: https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 10.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=e3a4e625f04c902327fcb3524a04db08b1a8b6e7e0656b3079312305ea6bf65e
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import pyspark
from pyspark import SparkContext
import math

In [ ]:
from google.colab import drive

## Monta um google drive

In [ ]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cd /content/gdrive/
!wget https://raw.githubusercontent.com/cscardoso/MC-CD02/main/2020_Yellow_Taxi_Trip_Data.csv -O /content/gdrive/MyDrive/yellow_tripdata_2020-01.csv



--2023-01-31 16:31:18--  https://raw.githubusercontent.com/cscardoso/MC-CD02/main/2020_Yellow_Taxi_Trip_Data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24162586 (23M) [text/plain]
Saving to: ‘/content/gdrive/MyDrive/yellow_tripdata_2020-01.csv’

/content/gdrive/MyD 100%[===================>]  23.04M  50.7MB/s    in 0.5s    

2023-01-31 16:31:19 (50.7 MB/s) - ‘/content/gdrive/MyDrive/yellow_tripdata_2020-01.csv’ saved [24162586/24162586]



In [ ]:
!ls -la /content/gdrive/MyDrive
!head /content/gdrive/MyDrive/yellow_tripdata_2020-01.csv

total 917695
-rw------- 1 root root       178 May 27  2021 '21ª Reunião -  07-05-21 - Integração do DJEnsemble ao sistema Gypscie (gqn-agdh-sue – 27 de mai. de 2021).gjam'
drwx------ 2 root root      4096 Jan  6  2022  Aluguel
drwx------ 2 root root      4096 Feb 28  2018  Anmily
drwx------ 2 root root      4096 Jan 29  2019  app
drwx------ 2 root root      4096 Jan 29  2020  Apresentacao_Petrobras
-rw------- 1 root root     28995 Nov 11 12:37 'Arquivos Radar.xlsx'
-rw------- 1 root root       178 Feb 21  2020 'assinado_Formulario para postulante a DAS-FCPE_anmilypaulamartins.gdoc'
-rw------- 1 root root    301807 Feb 21  2020 'assinado_Formulario para postulante a DAS-FCPE_anmilypaulamartins.pdf'
drwx------ 2 root root      4096 Jun 22  2020 'Audio Tereza'
-rw------- 1 root root       178 Sep 12  2017 'Automação exclusividade YARN.gdraw'
drwx------ 2 root root      4096 May  6  2020 'AVMC Pós-graduação'
-rw------- 1 root root       178 May  9  2022 'BOLETO_20908_19393497_07052

In [ ]:
spark = SparkContext(appName="aula-1", master="local[4]")

## Lê o arquivo no google drive
- O resultado cria um RDD com uma entrada por linha do arquivo
- um filtro retira a linha de cabeçalho:
VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge

In [ ]:
taxi_file_header = spark.textFile("/content/gdrive/MyDrive/yellow_tripdata_2020-01.csv")
taxi_file= taxi_file_header.filter(lambda l: not l.startswith("VendorID"))

In [ ]:
def testa_VendorId(id):
  if not id.isnumeric():
    return False
  else:  
    if (int(id) <= 20):
      return True
    else:
      return False    

## Gera um RDD onde cada item é extraido do arquivo a partir
## da delimitação por vírgula.
## Em seguida, filtra as entradas com VendorId <= 20

In [ ]:
print(taxi_file.count())
trip_column= taxi_file.map(lambda a: a.split(","))
print(trip_column.take(1))
list_trip = trip_column.filter(lambda a:testa_VendorId(a[0]))

246318
[['1', '01/01/2020 12:00:00 AM', '01/01/2020 12:13:03 AM', '1', '2.2', '1', 'N', '68', '170', '1', '10.5', '3', '0.5', '2.85', '0', '0.3', '17.15', '2.5']]


In [ ]:

print("Total de Linhas:"+str(list_trip.count()))

Total de Linhas:244306


## Criando pipelines sobre o RDD list_trip

# Spark Dataflow

1.   map
2.   groupBy
3.   MapValues








In [ ]:

transformIntRDD = list_trip.map(lambda a: (a[0],float(a[11])))
transformRDD = transformIntRDD.groupByKey().mapValues(lambda v:sum(v))

In [ ]:
transformRDD.take(2)

[('1', 180107.5), ('2', 37003.42000000004)]


# Projeta: VendorId, Numero de passageiros, Valores de Tarifa 



In [ ]:
projectedRDD = list_trip.map(lambda a: (a[0],a[3],a[11]))




## DataFlow: Calcula viagens prejuizo



1.   filter
2.   map





In [ ]:
print(projectedRDD.take(10))
print(projectedRDD.count())
tarifa_altaRDD = projectedRDD.filter(lambda a: float(a[2]) >5 and int(a[1])>3)
print(tarifa_altaRDD.count())

[('1', '1', '3'), ('2', '5', '0.5'), ('2', '5', '3'), ('2', '5', '0.5'), ('2', '5', '0.5'), ('2', '5', '0.5'), ('2', '5', '0.5'), ('2', '1', '0.5'), ('1', '2', '3'), ('1', '0', '3')]
244306
0


In [ ]:
prejuizo=tarifa_altaRDD.count()

In [ ]:
print("Viagens Prejuizo: "+str(prejuizo))

Viagens Prejuizo: 0


In [ ]:
!jps

2288 SparkSubmit
2756 Jps
